In [ ]:
import os
from glob import glob
import pickle

import numpy as np
import scipy
import scipy.stats

In [ ]:
# TODO: uncomment only two experiments
EXPERIMENTS = (
#     '0_baseline',
#     '1_mixup',
#     '2_mixup_nowd',
#     '3_uda1',
#     '4_uda2',
#     '5_uda1_mixup_nowd'
)

# TODO: uncomment the section related to the selected evaluation dataset
# if True:
#     DATASET = 'oai_imo'
#     CLASSES = list(range(1, 5))
#     ATLAS = 'segm'
# if True:
#     DATASET = 'okoa'
#     CLASSES = (1, 2)
#     ATLAS = 'okoa'

# TODO: specify path to your `results` directory
path_results_root = ''

path_base = os.path.join(path_results_root,
                         (f'{EXPERIMENTS[0]}/logs_{DATASET}_test'
                          f'/cache_{DATASET}_test_{ATLAS}_volumew_paired.pkl'))
path_eval = os.path.join(path_results_root,
                         (f'{EXPERIMENTS[1]}/logs_{DATASET}_test'
                          f'/cache_{DATASET}_test_{ATLAS}_volumew_paired.pkl'))

with open(path_base, 'rb') as f:
    dict_base = pickle.load(f)

with open(path_eval, 'rb') as f:
    dict_eval = pickle.load(f)

In [ ]:
kl_ranges = {
    '0': (0, ),
    '1': (1, ),
    '2': (2, ),
    '3': (3, ),
    '4': (4, ),
    'all': (0, 1, 2, 3, 4),
}

In [ ]:
dsc_base = np.asarray(dict_base['dice_score'])
dsc_eval = np.asarray(dict_eval['dice_score'])

for kl_name, kl_values in kl_ranges.items():
    sel_base = np.isin(np.asarray(dict_base['KL']), kl_values)
    sel_eval = np.isin(np.asarray(dict_eval['KL']), kl_values)
    assert np.all(np.equal(sel_base, sel_eval))
    sel = sel_base
    if not np.sum(sel):
        continue
    
    print(f'------ KL: {kl_name} ------')

    print('--- Wilcoxon signed-rank, two-sided ---')
    res_2side = [scipy.stats.wilcoxon(dsc_base[sel, c],
                                      dsc_eval[sel, c])
                 for c in CLASSES]
    print(*res_2side, sep='\n')

    print('--- Wilcoxon signed-rank, one-sided, less ---')
    res_1side = [scipy.stats.wilcoxon(dsc_base[sel, c],
                                      dsc_eval[sel, c],
                                      alternative='less')
                 for c in CLASSES]
    print(*res_1side, sep='\n')

    print('--- Wilcoxon signed-rank, one-sided, greater ---')
    res_1side = [scipy.stats.wilcoxon(dsc_base[sel, c],
                                      dsc_eval[sel, c],
                                      alternative='greater')
                 for c in CLASSES]
    print(*res_1side, sep='\n')